### BERT 모델을 활용한 네이버 영화 리뷰 예측하기

All reviews are shorter than 140 characters
- Each sentiment class is sampled equally (i.e., random guess yields 50% accuracy)
- 100K negative reviews (originally reviews of ratings 1-4)
- 100K positive reviews (originally reviews of ratings 9-10)
- Neutral reviews (originally reviews of ratings 5-8) are excluded

In [41]:
!pip install transformers

In [42]:
!pip install utils

In [43]:
from collections import defaultdict
from glob import glob
from bs4 import BeautifulSoup
from lxml import html
import os
import re
import time
import numpy as np
import pandas as pd
import requests
import utils
import requests
import pandas as pd
import urllib.parse
import time
import torch
from keras.preprocessing.sequence import pad_sequences

In [44]:
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split

In [45]:
import tensorflow as tf

In [46]:
BASEURL     = 'http://movie.naver.com/movie/point/af/list.nhn'
RATINGURL   = BASEURL + '?&page=%s'
MOVIEURL    = BASEURL + '?st=mcode&target=after&sword=%s&page=%s'
DATADIR     = 'data/ratings'
INDEXFILE   = 'index.txt'
TMPFILE     = 'data/ratings_all.txt'
RATINGSFILE = 'data/ratings.txt'

SEED        = 1234
SLEEP       = 600
NDOCS       = 200000

extract_nums = lambda s: re.search('\d+', s).group(0)
sanitize_str = lambda s: s.strip()

In [47]:
source = BeautifulSoup(requests.get(BASEURL).content, 'html.parser')
alltables = source.find_all("table")
table = source.find("table", {"id":"thetable"})


In [48]:
moviecomments = pd.DataFrame(columns=['ID', 'TITLE', 'POINT', "COMMENT"])
data = []

for page in range(1, 10):

    RATINGURL = BASEURL + '?&page='+str(page)
    if page % 100==0:
      print(RATINGURL, len(data))
    source = BeautifulSoup(requests.get(RATINGURL).content, 'html.parser')
    
    for tr in source.find_all('tr')[1:]:
       
        try:
            tds = tr.find_all('td')
            iid = tds[0].text
            title = tds[1].find("a").text
            point = tds[1].find("div").text.replace("\n별점 - 총 10점 중", "").replace("\n", "")
            contents = tds[1].text.strip().split('\n')[4]
            data.append([iid, title, point, contents])
            continue
        except:
            print("오류")
    time.sleep(0.1)
    
moviecomments = pd.concat([moviecomments, pd.DataFrame(data, columns=['ID', 'TITLE', 'POINT', "COMMENT"])])
moviecomments.to_csv("naverfile.csv", sep=",")

In [50]:
moviecomments = pd.read_csv("naverfile.csv", sep=",")

점수가 숫자가 아닌 경우 제외

In [51]:
moviecomments = moviecomments[moviecomments['POINT'].isin(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])]

In [52]:
moviecomments['POINT'] = moviecomments['POINT'].astype(str).astype(int)
moviecomments['Y'] = -1
moviecomments.loc[moviecomments['POINT']>=9, "Y"] = 1
moviecomments.loc[moviecomments['POINT']<=4, "Y"] = 0

### 중립은 제거, 긍정과 부정 리뷰만 데이터로 사용

In [53]:
moviecomments = moviecomments[moviecomments['Y']>=0]

In [54]:
moviecomments.head()

,Unnamed: 0,ID,TITLE,POINT,COMMENT,Y
0,0,16934629,#살아있다,4,배우 아니였음 안봤다..여자 작가님이신가... 연기랑 배우만 있고 왜에 대한 생각 ...,0
1,1,16934628,트루먼 쇼,10,진짜 볼때마다 충격이 어마어마한 영화. 트루먼이 나오자 환호하던 사람들마저 다른덴 ...,1
2,2,16934627,#살아있다,1,"끊었던 담배 생각이,,,,ㅠㅠ 반도를 기대해봅니당",0
4,4,16934625,어드리프트:우리가 함께한 바다,10,갑자기 생각나서 다시 보게 됐는데 다시 봐도 가슴이 아리네...ㅠㅠ,1
5,5,16934624,태백산맥,10,조정래 선생의 역작... 슬픈 현대사... 끝나지 않았다...,1


### 리뷰 문장 추출

In [55]:
sentences = moviecomments['COMMENT']
sentences[:10]

0     배우 아니였음 안봤다..여자 작가님이신가... 연기랑 배우만 있고 왜에 대한 생각 ...
1     진짜 볼때마다 충격이 어마어마한 영화. 트루먼이 나오자 환호하던 사람들마저 다른덴 ...
2                          끊었던 담배 생각이,,,,ㅠㅠ 반도를 기대해봅니당 
4                갑자기 생각나서 다시 보게 됐는데 다시 봐도 가슴이 아리네...ㅠㅠ 
5                   조정래 선생의 역작... 슬픈 현대사... 끝나지 않았다... 
6                        이 영화는 제 영화순위중 최고로 높은 제 인생작입니다 
7                   중간에 화장실 가신분들 안 돌아왔어요무슨일 생기신거 아니겠죠? 
8                            킬링타임용!! 시간많이 남으시는 분들은 보세요 
9                           자나깨나 운전조심.한순간에  조상님만날수있음ㅡㅡ 
10       제발 이 영화 망하면 소원이 없겠다. 다시는 영화 찍을 생각 하지마 이 기집애야. 
Name: COMMENT, dtype: object

In [56]:
 #BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] 배우 아니였음 안봤다..여자 작가님이신가... 연기랑 배우만 있고 왜에 대한 생각 일도 안하나보다..ㅜㅜ  [SEP]',
 '[CLS] 진짜 볼때마다 충격이 어마어마한 영화. 트루먼이 나오자 환호하던 사람들마저 다른덴 뭐하지 하며 쇼로 취급하던 사람들을 보고 많은 생각이 들었다.. 평생 열린결말일 트루먼을 응원하며...  [SEP]',
 '[CLS] 끊었던 담배 생각이,,,,ㅠㅠ 반도를 기대해봅니당  [SEP]',
 '[CLS] 갑자기 생각나서 다시 보게 됐는데 다시 봐도 가슴이 아리네...ㅠㅠ  [SEP]',
 '[CLS] 조정래 선생의 역작... 슬픈 현대사... 끝나지 않았다...  [SEP]',
 '[CLS] 이 영화는 제 영화순위중 최고로 높은 제 인생작입니다  [SEP]',
 '[CLS] 중간에 화장실 가신분들 안 돌아왔어요무슨일 생기신거 아니겠죠?  [SEP]',
 '[CLS] 킬링타임용!! 시간많이 남으시는 분들은 보세요  [SEP]',
 '[CLS] 자나깨나 운전조심.한순간에  조상님만날수있음ㅡㅡ  [SEP]',
 '[CLS] 제발 이 영화 망하면 소원이 없겠다. 다시는 영화 찍을 생각 하지마 이 기집애야.  [SEP]']

In [57]:
# 라벨 추출
labels = moviecomments['Y'].values
labels

array([0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 0])

영화리뷰 긍정비율

In [58]:
len(moviecomments[moviecomments['Y']==1])/len(moviecomments['Y'])

0.4583333333333333

In [59]:
moviecomments['Y'].value_counts()

0    39
1    33
Name: Y, dtype: int64

In [60]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] 배우 아니였음 안봤다..여자 작가님이신가... 연기랑 배우만 있고 왜에 대한 생각 일도 안하나보다..ㅜㅜ  [SEP]
['[CLS]', '배우', '아', '##니', '##였', '##음', '안', '##봤', '##다', '.', '.', '여자', '작', '##가', '##님', '##이', '##신', '##가', '.', '.', '.', '연', '##기', '##랑', '배우', '##만', '있고', '왜', '##에', '대한', '생', '##각', '일', '##도', '안', '##하', '##나', '##보다', '.', '.', '[UNK]', '[SEP]']


In [61]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,  84703,   9519,  25503, 119147,  32158,   9521, 118991,
        11903,    119,    119,  62592,   9652,  11287, 108578,  10739,
        25387,  11287,    119,    119,    119,   9568,  12310,  62200,
        84703,  19105,  40523,   9596,  10530,  18154,   9420,  66540,
         9641,  12092,   9521,  35506,  16439,  80001,    119,    119,
          100,    102,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [62]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [63]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=2020, 
                                                                                    test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2020, 
                                                       test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

tensor([   101,   8924, 118729,   9365,  21386,  25549,  12674, 119140,  14040,
         15184,   9715,  38696,  12310,    119,    119,   9764, 119110,  10459,
         30858,  18227,    102,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0, 

In [64]:
# 데이터를 파이토치의 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

tensor([   101,  84703,   9519,  25503, 119147,  32158,   9521, 118991,  11903,
           119,    119,  62592,   9652,  11287, 108578,  10739,  25387,  11287,
           119,    119,    119,   9568,  12310,  62200,  84703,  19105,  40523,
          9596,  10530,  18154,   9420,  66540,   9641,  12092,   9521,  35506,
         16439,  80001,    119,    119,    100,    102,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0, 

In [65]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)




In [66]:
print("학습 데이터수:", len(train_dataloader))
print("검증 데이터수:", len(validation_dataloader))
print("테스트 데이터수:", len(test_dataloader))


학습 데이터수: 2
검증 데이터수: 1
테스트 데이터수: 3


### 모델생성

In [67]:
# # GPU 디바이스 이름 구함
# device_name = tf.test.gpu_device_name()

# # GPU 디바이스 이름 검사
# if device_name == '/device:GPU:0':
#     print('Found GPU at: {}'.format(device_name))
# else:
#     raise SystemError('GPU device not found')

In [68]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')


No GPU available, using the CPU instead.


In [69]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
#model.cuda()

In [70]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 4

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 학습률을 조금씩 감소시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

### 모델 학습

In [71]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [72]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [73]:
import random
import datetime

In [82]:
# 재현을 위해 랜덤시드 고정
seed_val = 0
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


In [75]:
# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행     
        b_input_ids = b_input_ids.cpu().long()
        b_input_mask = b_input_mask.cpu().long()
        b_input_ids = b_input_ids.cpu().long()
        
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...

  Average training loss: 0.71
  Training epcoh took: 0:04:26

Running Validation...
  Accuracy: 0.50
  Validation took: 0:00:11

======== Epoch 2 / 4 ========
Training...

  Average training loss: 0.68
  Training epcoh took: 0:04:24

Running Validation...
  Accuracy: 0.75
  Validation took: 0:00:11

======== Epoch 3 / 4 ========
Training...

  Average training loss: 0.67
  Training epcoh took: 0:04:25

Running Validation...
  Accuracy: 0.75
  Validation took: 0:00:11

======== Epoch 4 / 4 ========
Training...

  Average training loss: 0.67
  Training epcoh took: 0:04:24

Running Validation...
  Accuracy: 0.88
  Validation took: 0:00:11

Training complete!


### 테스트셋 평가

In [76]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))


Accuracy: 0.73
Test took: 0:01:37


In [80]:
logits = test_sentences(['주연배우가 아깝다. 총체적 난국...'])

print(logits)
print(np.argmax(logits))

[[ 0.35163674 -0.07119612]]
0
